# Correct Wave III

We have to update (overwrite) some rows in the database, based on a new .mbd file that the Sardinians sent us.

```
Marco Masala <marco.masala@gmail.com>
Monday, July 3, 2017 at 12:55 PM
Dear Christopher,
I updated the third wave database with the data provided by Angelo Scuteri. The only table that has been modified is VASCULAR where new records have been added. New records contain information only on the diam_S, diam_D, and IMT fields.
```

# Initial declarations

In [1]:
pwd

'/Users/colettace/projects/david/sardiNIA_database/latest/20170703_IMT_update'

In [2]:
import pandas as pd
from subprocess import check_output
from io import BytesIO

In [3]:
pd.__version__

'0.20.2'

In [5]:
prf = pd.read_csv( '../TSV_release/2017-02-24-Sardinia-Data-TAB.txt', sep='\t')

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (20,21,22,23,24,30,72,73,76,77,80,81,101,105,106,114,166,203,340,750) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Load Sardinia's Microsoft Access Files that contains the correction

In [6]:
new_db_path = 'dbSardiniaIII_2017_vascular_update.mdb'

## Get Names of tables in mdb file

In [7]:
table_names = check_output( ['mdb-tables', new_db_path]).decode().strip().split()

In [8]:
len(table_names)

31

In [9]:
table_names

['BLOOD',
 'DISEASES',
 'DISEASESCHECKLIST_n',
 'ECG',
 'ECOCARDIO_ANAMNESI',
 'ECOCARDIO_Aortic_Valve',
 'ECOCARDIO_AVsuLA',
 'ECOCARDIO_common',
 'ECOCARDIO_Mitral_Valve',
 'ECOCARDIO_Tricuspid_Valve',
 'EXAMINATION_BIS',
 'HOLTER',
 'INDIVIDUAL',
 'MEDHISTORY',
 'MMSE',
 'OTHERMEDICATIONS_2',
 'OTHERMEDICATIONS1',
 'PHYSICAL_ACTIVITY',
 'PHYSICAL_TEST',
 'SPHIGMOCOR',
 'SPIROMETER',
 'URINE',
 'VASCULAR',
 'DISEASECHECKLIST',
 'ECOCARDIO_LV_Teichholz_M',
 'MEDICATIONS',
 'PQCT',
 'CESD',
 'EXAMINATION',
 'NEO',
 'PVW']

# Load Italian Data

Take only the table that contains the corrected fields, i.e., the "VASCULAR" table. (I'm reusing this code from a notebook that usually loads all data from all tables, thus the seemingly unnecesary <code>for</code> loop)

In [10]:
print ('\t'.join( ["Nrows", "Nindivs", "Nfields", "TableName" ] ))

all_tables = {}

for table_name in table_names:
    
    if table_name != 'VASCULAR':
        continue
    # Pull the table
    df = pd.read_csv( BytesIO( check_output( ['mdb-export', new_db_path, table_name ] ) ) )
    
    # how many unique individual ids in this table?
    try:
        individual_id_set = set( df.ID_INDIVIDUAL )
    except AttributeError:
        individual_id_set = set( df.id_individual )
    
    # Get table shape
    rows, cols = df.shape

    print ("{}\t{}\t{}\t{}".format( rows, len( individual_id_set ), cols, table_name ))

    all_tables[ table_name ] = df

Nrows	Nindivs	Nfields	TableName
4842	4842	13	VASCULAR


In [11]:
len(all_tables)

1

# Get statistics from new data

We know from Marco's email reprinted above that there are only three fields that we care about, namely, 'diam_D', 'diam_S', 'IMT'.

The <code>.count()</code> function counts the number of non-empty cells.

In [12]:
incoming_vascular = all_tables['VASCULAR'].loc[ :, ['ID_INDIVIDUAL', 'diam_D', 'diam_S', 'IMT'] ]

In [13]:
incoming_vascular.columns = ['id_individual', 'diam_D', 'diam_S', 'IMT']

In [14]:
incoming_vascular.count()

id_individual    4842
diam_D           2337
diam_S           2270
IMT              4404
dtype: int64

# Get statistics from old data

Note that the field names from the Italian and US databases for the same variable are different. There is an Excel spreadsheet in the FTP that shows the correspondance between the two.

In [15]:
existing_vascular = prf.loc[ prf.Wave == 3, ['Wave', 'id_individual', 'vasDiaDiam', 'vasSysDiam', 'vasIMT' ] ]

In [16]:
existing_vascular.head()

,Wave,id_individual,vasDiaDiam,vasSysDiam,vasIMT
5,3,7,6.1,6.5,0.61
12,3,9,NaN,NaN,NaN
19,3,13,NaN,NaN,NaN
23,3,14,NaN,NaN,0.62
25,3,17,NaN,NaN,NaN


In [17]:
existing_vascular.count()

Wave             4842
id_individual    4842
vasDiaDiam        634
vasSysDiam        614
vasIMT           2219
dtype: int64

As you can see there are vastly  more empty cells in our public release version than the ones that Marco sent.

# Are we talking about the same people?

In [18]:
set( existing_vascular.id_individual ) == set( incoming_vascular.id_individual )

True

# Segregate database into wave 3/non wave 3

In [19]:
wave3prf = prf.loc[ prf.Wave == 3 ]

In [20]:
nonwave3prf = prf.loc[ prf.Wave != 3 ]

# Check to see there aren't multiple records for participant

In [21]:
from collections import Counter

In [22]:
[ _id for _id, count in Counter( list(wave3prf.id_individual)).items() if count > 1]

[]

In [23]:
[ _id for _id, count in Counter( list(incoming_vascular.id_individual)).items() if count > 1]

[]

They are unique individuals!

# Make row indices be the value in id_individual

I learned this the hard way: if you're going to update/overwrite some cells, they *MUST* have the same index and column headers. Before, in the NIH public release database, the row index was an arbitrary integer. We'll temporarily assign the value of the <code>id_individual</code> field to be the new index, so that we can do the update operation. Then when we're done, we'll reassign the old index back to the rows and rejoin then with the data from the other waves.

Warning: the <code>set_index()</code> function by default yields a new object, so use the <code>inplace=True</code> argument.

It is shown that the destination cells are already sorted by id_individual. Sort the incoming rows by index to match (although technically I think it's unnecessary to do that, pretty sure PANDAS will do the update operation correctly as long as the row indices are correct, they can be out of order.)

In [24]:
original_index = wave3prf.id_individual

In [25]:
wave3prf.set_index( 'id_individual', inplace=True, drop=False )

In [26]:
incoming_vascular.set_index( 'id_individual', inplace=True, drop=True )

In [27]:
incoming_vascular.sort_index(inplace=True)

In [28]:
all(wave3prf.index == incoming_vascular.index)

True

# Check the target cells before asignment

In [29]:
wave3prf.loc[ :, ['vasDiaDiam', 'vasSysDiam', 'vasIMT' ] ].count()

vasDiaDiam     634
vasSysDiam     614
vasIMT        2219
dtype: int64

# Make the assignment

For some reason, the update operation works using the syntax in the following cell, but does *NOT* work using the <code>.loc[]</code> syntax like they keep telling everyone to use:

In [30]:
wave3prf[ ['vasDiaDiam', 'vasSysDiam', 'vasIMT' ] ] = \
    incoming_vascular[ ['diam_D', 'diam_S', 'IMT'] ]

/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


# Check to see it worked

In [31]:
wave3prf[ ['vasDiaDiam', 'vasSysDiam', 'vasIMT' ] ].count()

vasDiaDiam    2337
vasSysDiam    2270
vasIMT        4404
dtype: int64

In [32]:
wave3prf.index = original_index.index

# Merge back with the rest of the waves

In [34]:
prf.loc[ prf.Wave == 3, ['vasDiaDiam', 'vasSysDiam', 'vasIMT' ] ].count()

vasDiaDiam     634
vasSysDiam     614
vasIMT        2219
dtype: int64

In [35]:
prf = pd.concat( [nonwave3prf, wave3prf] ).sort_index()

In [36]:
prf.loc[ prf.Wave == 3, ['vasDiaDiam', 'vasSysDiam', 'vasIMT' ] ].count()

vasDiaDiam    2337
vasSysDiam    2270
vasIMT        4404
dtype: int64

# Database cleanup item 2:

This never got done in this release. Save it for next time:

```
From: "Qian, Yong (NIH/NIA/IRP) [C]" <QianY@grc.nia.nih.gov>
Date: Friday, April 28, 2017 at 12:08 PM
To: "Coletta, Christopher (NIH/NIA/IRP) [E]" <christopher.coletta@nih.gov>
Cc: "Ding, Jun (NIH/NIA/IRP) [E]" <jun.ding@nih.gov>, "Schlessinger, David (NIH/NIA/IRP) [E]" <SchlessingerD@grc.nia.nih.gov>
Subject: repeated records in 2017-02-24-Sardinia-Data-TAB.txt
 
Hi Chris,

There were a few wave2 pwv records that are repeated in 2017-02-24-Sardinia-Data-TAB.txt.  Can you correct the database file?
I understand that the you didn’t process the wave2 data so it was not your fault.
 
Here are those:
id_individual   Wave    pwvDate pwvQual pwv
3528    2       2006-01-27      3       509
4285    2       2005-06-10      3       754.8   
11633   2       2006-12-06      3       590.5   
27393   2       2006-09-13      3       957.5   
30245   2       2006-01-31      2       771.7   
 
Yong
```

======

```
From: Coletta, Christopher (NIH/NIA/IRP) [E]
Sent: Friday, April 28, 2017 12:12:24 PM
To: Qian, Yong (NIH/NIA/IRP) [C]
Cc: Ding, Jun (NIH/NIA/IRP) [E]; Schlessinger, David (NIH/NIA/IRP) [E]
Subject: Re: repeated records in 2017-02-24-Sardinia-Data-TAB.txt
 
I understand.
 
When the same thing happened in the Wave IV data (duplicate PWV readings), I looked at the date and selected the ones the fell within the Wave IV date range. I’ll try to do the same in this case.
 
Is there a table somewhere available showing the dates that correspond to the waves?
 
-Chris
 ```
 
======

 ```
 I don't think there is a date table for waves.  

I think in general, waves are defined as:
wave1:  12/2001 to 5/2004
wave2:    5/2004 to 7/2008
wave3:    7/2008 to 1/2012
wave4:    2/2012 to 5/2016 ?

correct me if I am wrong.

Yong
```

# Format the database for text/TSV format and write to file

Convert all numeric values to strings and lop off .0's, etc

In [38]:
for name in list(prf.columns):
    prf[name] = prf[name].astype(str)
prf.replace( to_replace='nan', value='', inplace=True, regex=False )
prf.replace( to_replace='NaT', value='', inplace=True, regex=False )
prf.replace( regex=True, inplace=True, to_replace=r'\.0$', value='')
prf.replace( regex=True, inplace=True, to_replace=r'00000+\d$', value='')
prf.replace( regex=True, inplace=True, to_replace=r'999999+\d$', value='')

In [39]:
prf.to_csv( '2017-07-07-Sardinia-Data-TAB.txt', sep='\t', encoding='utf-8', index=False)

In [41]:
ls -l *TAB*.txt

-rw-r--r--@ 1 colettace  NIH\Domain Users  41187887 Jul  3 16:22 2017-07-03-Sardinia-TAB.txt
-rw-r--r--  1 colettace  NIH\Domain Users  41217915 Jul  7 16:54 2017-07-07-Sardinia-Data-TAB.txt
-rw-r--r--  1 colettace  NIH\Domain Users  41217915 Jul  7 18:00 2017-07-07-Sardinia-TAB-NEW.txt


In [40]:
len(prf)

20968

# Final Check to make sure everything is converted

Remember, everything is a string now, so look for wave '3' not wave 3.

In [43]:
prf.loc[ prf.Wave == '3', ['Wave', 'id_individual', 'vasDiaDiam', 'vasSysDiam', 'vasIMT' ] ].head()

,Wave,id_individual,vasDiaDiam,vasSysDiam,vasIMT
5,3,7,6.1,6.5,0.61
12,3,9,,,0.44
19,3,13,7.2,7.7,0.84
23,3,14,,,0.62
25,3,17,4.9,5.5,0.59


# External checking

```diff <(cut -d$'\t' -f474 2017-07-07-Sardinia-TAB.txt ) <(cut -d$'\t' -f474 ../2017-02-24-Sardinia-TAB.txt)```

In [44]:
incoming_vascular.head()

,diam_D,diam_S,IMT
id_individual,,,
7,6.1,6.5,0.61
9,NaN,NaN,0.44
13,7.2,7.7,0.84
14,NaN,NaN,0.62
17,4.9,5.5,0.59


In [45]:
prf.loc[ prf.Wave == '3', ['Wave', 'id_individual', 'vasDiaDiam', 'vasSysDiam', 'vasIMT' ] ].tail(20)

,Wave,id_individual,vasDiaDiam,vasSysDiam,vasIMT
20886,3,92084,,,0.42
20888,3,92085,,,0.47
20890,3,92086,,,0.46
20892,3,92093,,,0.55
20893,3,92119,,,0.55
20895,3,92205,,,0.67
20897,3,92253,,,0.43
20899,3,92281,,,0.56
20901,3,92318,5.9,6.2,0.68
20903,3,92320,5.3,5.9,0.65


In [46]:
incoming_vascular.tail(20)

,diam_D,diam_S,IMT
id_individual,,,
92084,NaN,NaN,0.42
92085,NaN,NaN,0.47
92086,NaN,NaN,0.46
92093,NaN,NaN,0.55
92119,NaN,NaN,0.55
92205,NaN,NaN,0.67
92253,NaN,NaN,0.43
92281,NaN,NaN,0.56
92318,5.9,6.2,0.68
